In [10]:
from models import LatentClassifier, VAE_CELL_CNN

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# mnist dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms

from dataset_tools import OwnDataset, make_train_test_val_split
#from plotting import show_img, loss_plots, accuracy_plots, plot_random_images, plot_image_comparison, plot_latent
from collections import defaultdict


from statsmodels.stats.contingency_tables import mcnemar
from models import LatentClassifier, VAE_CELL_CNN, CELL_CNN_CLASSIFIER, CELL_CNN_CLASSIFIER_2


cuda = torch.cuda.is_available()
print(cuda)

False


# Loading Data 

In [49]:
val = True
tf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    # transforms.Lambda(lambda x: x.view(-1)) # notice that we dont flatten when we are going to use CNN
])
dataset = OwnDataset(transform=tf, path = r"C:\Users\vlbr1\Downloads\labelled_data-20240618T120814Z-001\labelled_data")
# dataset = OwnDataset(transform=tf)
train_subset, test_subset, val_subset = make_train_test_val_split(dataset)


train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=True)

print(len(dataset), len(train_loader), len(test_loader), len(val_loader))



2684 27 9 7


# Loading Models 

In [45]:

from models import LatentClassifier_2, VAE_CELL_CNN, CELL_CNN_CLASSIFIER, CELL_CNN_CLASSIFIER_2, VAE_CELL_CNN
from loss_functions import loss_function
# import mse loss


# now we load the saved models into the model class
MODEL_DIR = "trained_models/"


latent_dim_256 = LatentClassifier_2(hidden_dim=512, latent_dim=256, num_classes=13)
latent_dim_256.load_state_dict(torch.load(MODEL_DIR + f"{latent_dim_256.__class__.__name__}_latent{latent_dim_256.latent_dim}_mean_best_model.pth",map_location=torch.device('cpu')))

latent_dim_2 = LatentClassifier_2(hidden_dim=4, latent_dim=2, num_classes=13)
latent_dim_2.load_state_dict(torch.load(MODEL_DIR + f"{latent_dim_2.__class__.__name__}_latent{latent_dim_2.latent_dim}_mean_best_model.pth",map_location=torch.device('cpu')))

Cell_CNN_Classifier = CELL_CNN_CLASSIFIER(input_dim=(3,68,68),hidden_dim=128, num_classes=13)
Cell_CNN_Classifier.load_state_dict(torch.load(MODEL_DIR + f"{Cell_CNN_Classifier.__class__.__name__}_best_model.pth",map_location=torch.device('cpu')))

Vae_Cell_CNN_256 = VAE_CELL_CNN(input_dim=(3,68,68), hidden_dim=512, latent_dim=256)
Vae_Cell_CNN_256.load_state_dict(torch.load(MODEL_DIR + f"{Vae_Cell_CNN_256.__class__.__name__}_latent{Vae_Cell_CNN_256.latent_dim}_mean_best_model.pth",map_location=torch.device('cpu')))


Vae_Cell_CNN_2 = VAE_CELL_CNN(input_dim=(3,68,68), hidden_dim=4, latent_dim=2)
Vae_Cell_CNN_2.load_state_dict(torch.load(MODEL_DIR + f"{Vae_Cell_CNN_2.__class__.__name__}_latent{Vae_Cell_CNN_2.latent_dim}_mean_best_model.pth",map_location=torch.device('cpu')))


# models = [vae_cnn_own]
models = [latent_dim_256, latent_dim_2, Cell_CNN_Classifier]
# models = [ae_cnn_own]



Comparing the classification performance across trained models

In [47]:
batch_accuracy_256_latent = []
batch_accuracy_2_latent = []
batch_accuracy_CNN = []
for image, moa in train_loader:
    print(Cell_CNN_Classifier(image))
    print(moa)
    print("_________")

    
    # for 256 dim latent space
    latent_output = Vae_Cell_CNN_256(image)
    x_hat, mu, sigma = latent_output["x_hat"], latent_output["mu"], latent_output["sigma"]

    pred_256 = latent_dim_256(mu)
    batch_accuracy_256_latent.append(pred_256.argmax(1)== moa)


    # for 2 dim latent space
    latent_output = Vae_Cell_CNN_2(image)
    x_hat, mu, sigma = latent_output["x_hat"], latent_output["mu"], latent_output["sigma"]

    pred_2 = latent_dim_2(mu)
    batch_accuracy_2_latent.append(pred_2.argmax(1)== moa)


    #Cell_CNN_Classifier(image) == moa
    batch_accuracy_CNN.append((Cell_CNN_Classifier(image).argmax(1) == moa))



    print(batch_accuracy_CNN)
    print(batch_accuracy_256_latent)
    print(batch_accuracy_2_latent)
    break
    
    
    
 

tensor([[-8.4337e+00,  1.1878e+00, -4.4942e+00,  3.3506e+00,  9.1874e-01,
         -2.8635e+00, -2.5352e+00, -2.8259e+00, -4.1474e+00, -4.0300e+00,
         -1.4942e-01, -1.0953e+00, -1.3986e+00],
        [-6.8659e+00, -6.3599e-01, -1.8447e+00,  1.0988e-01, -2.0516e+00,
         -1.0632e+00, -5.3426e-01, -3.5955e-01, -6.6403e+00,  1.0574e+00,
         -2.2186e-01, -1.7064e+00,  3.9914e+00],
        [-1.3475e+01,  3.5913e-01, -8.1337e+00,  6.3184e+00,  3.2303e+00,
         -5.8279e+00, -5.2161e+00, -6.2561e-01, -3.7365e+00, -8.9664e+00,
         -3.4720e+00, -6.0855e+00, -2.2034e+00],
        [-4.4871e+00,  1.0794e-01, -6.6241e+00,  1.4004e+00, -3.0188e-01,
         -1.9427e+00, -7.2265e-01, -1.8235e-01, -6.3688e+00, -3.5282e+00,
          1.8347e+00, -1.3660e+00,  5.8264e+00],
        [-1.2372e+01, -6.4223e-03, -6.0245e+00,  5.0577e+00,  1.8081e+00,
         -5.1742e+00, -3.6625e+00, -7.9913e-01,  1.3976e+00, -5.9153e+00,
         -1.0790e+00, -3.6217e+00, -1.4990e+00],
        [-1.339

# Confusion matrix 

In [ ]:
model_A_predictions = 
model_B_predictions = 
true_labels =

# cheking if length is the same 
assert len(model_A_predictions) == len(model_B_predictions) == len(true_labels)

In [ ]:
# contingency table 
n00 = n01 = n10 = n11 = 0

for true, pred_A, pred_B in zip(true_labels, model_A_predictions, model_B_predictions):
    if pred_A == true and pred_B == true:
        n00 += 1
    elif pred_A == true and pred_B != true:
        n01 += 1
    elif pred_A != true and pred_B == true:
        n10 += 1
    else:
        n11 += 1

table = [[n00, n01], [n10, n11]]

table

# Macnemar test + p-value 

In [ ]:
result = mcnemar(table, exact=True)
print(f'statistic={result.statistic}, p-value={result.pvalue}')

alpha = 0.05
if result.pvalue < alpha:
    print('Significant difference in performance (reject H0)')
else:
    print('No significant difference in performance (fail to reject H0)')